In [1]:
import pandas as pd
import json
import ast
import configparser
import pymysql

In [2]:
def pipeline(path):
    """
    Function performs required steps to:
    1) clean dataframe
    2) add ingestion timestamp column
    3) prepare dataframe data to insert to MySQL database
    """
    df = pd.read_csv(path)
    df.fillna('', inplace=True)
    df["ingestion_ts"] = pd.to_datetime(pd.Timestamp.today())
    df = df.values.tolist()
    
    return df

In [3]:
path = 'data/ready_for_ingestion/genres.csv'
genres_to_insert = pipeline(path)

In [4]:
preview = pd.read_csv(path)

In [5]:
parser = configparser.ConfigParser()
parser.read("config\mysql.conf")
hostname = parser.get("mysql_config", "hostname")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
dbname = parser.get("mysql_config", "database")
port = parser.get("mysql_config", "port")

conn = pymysql.connect(
    host=hostname,
    user=username,
    password=password,
    db=dbname,
    port=int(port)
)

if conn is None:
    print("Error. Connection to MySQL cannot be established.")
else:
    print("Successfully connected to MySQL.")

cur = conn.cursor()

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] Nie można nawiązać połączenia, ponieważ komputer docelowy aktywnie go odmawia)")

In [ ]:
create_table = """
CREATE TABLE IF NOT EXISTS movies_raw.genres (
    genre_id INT, 
    genre_name VARCHAR(255),
    movie_id INT,
    ingestion_ts TIMESTAMP
);
"""

insert_query = """
INSERT INTO movies_raw.genres (genre_id, genre_name, movie_id, ingestion_ts)
VALUES (%s, %s, %s, %s);
"""

In [ ]:
cur.execute(create_table)

In [ ]:
# Wstawienie danych do tabeli
cur.executemany(insert_query, genres_to_insert)

# Zatwierdzenie zmian w bazie danych
conn.commit()

# Zamknięcie kursora i połączenia
cur.close()
conn.close()